# Chapter 2: The Basics

In [16]:
int i1 = 1;
int i2{1};
auto i3{1.2};

cout << i1 << ' ' << i2 << ' ' << i3 << endl;

1 1 1.2


In [30]:
double sum(const vector<double>&v) { 
    double s = 0;
    for (double x : v) {
        s += x;
    }
    return s;
}

In [32]:
constexpr double square(double x) { return x * x; }

 `const` : value can not be changed.
 

 `constexpr`: to be valued at compile time.

In [35]:
const int a = 1;
int b = 2;
constexpr double c1 = 1.4 * a;
// constexpr double c2 = 1.4 * b; // error

vector<double> v {1.2, 3.4, 4.5};
const double s1 = sum(v);
cout << s1 << endl;

// constexpr double s2 = sum(v); // error
constexpr double s3 = square(a);


9.1


`*`:
* in declaration: pointer.
* otherwise: contents of.

`&`: 
* in declaration: reference to.
* otherwise: address of.

`.`: access `struct` members through a name or reference.

`->`: access `sturct` members through a pointer.


In [38]:
enum class Color { red, blue, green };

Color col = Color::red;


In [ ]:
namespace MyCode {
    class complex { };
    complex sqrt(complex);
    int main();
}

In [ ]:
static_assert(4 <= sizeof(int), "intergers are too small"); // check at compile time

# Chapter 3: Abstraction Mechanisms

`std::initializer_list`: define the initializer-list constructor

In [ ]:
Vector::Vector<std:initializer_list<double> lst):elem{new double[lst.size()], sz{lst.size()}} {
    copy(lst.begin(), lst.end(), elem);
}

In [41]:
class Point
{
    private:
        int x, y;
    public:
        Point() : x(0), y(0) {}
        Point(int x, int y) : x(x), y(y) {}
}

In [42]:
class Shape {
public:
    virtual Point center() const=0;
    virtual void move(Point to)=0;
    
    virtual void draw() const=0;
    virtual void rotate(int angle)=0;
    
    virtual ~Shape(){}
    
};

In [43]:
void rotate_all(vector<Shape*>& v, int angle) 
{
    for (auto p : v)
        p->rotate(angle);
}

In [44]:
class Circle : public Shape {
public:
    Circle(Point p, int rr);
    
    Point center() const { return x; }
    void move(Point to) { x = to; }
    
    void draw() const;
    void rotate(int) {}
    
private:
    Point x;
    int r;
};

In [45]:
class Smiley : public Circle {
public:
    Smiley(Point p, int r): Circle{p, r}, mouth{ nullptr } {}
    
    ~Smiley()
    {
        delete mouth;
        for (auto p : eyes) delete p;
    }
    
    void move(Point to);
    
    void draw() const;
    void rotate(int);
    
    void add_eye(Shape* s) { eyes.push_back(s); }
    void set_mouth(Shape* s);
    virtual void wink(int i);
    
private:
    vector<Shape*> eyes;
    Shape* mouth;
}

void Smiley::draw()
{
    Circle::draw();
    for (auto p : eyes)
        p->draw();
    mouth->draw();
}


* copy constructor: called when class was created.
* copy assignment：called when assigned. 

In [56]:
class A {
 public:
  A() {
    std::cout << "A() called!\n";
  }
  A(const A& a) {
    std::cout << "A(const A& a) called!\n";
  }
  A& operator=(const A& a) {
    std::cout << "operator=(const A& a)!\n";
    return *this;
  }
};


In [57]:
 A a1;
 A a2 = a1;
 A a3(a1);
 a3 = a2;

A() called!
A(const A& a) called!
A(const A& a) called!
operator=(const A& a)!
